<a href="https://colab.research.google.com/github/Melvinmcrn/Project_ML_Soccer-world-cup-2018/blob/master/Soccer%20World%20Cup%202018%20Winner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd 
from matplotlib import pyplot as plt
import seaborn as sb
import numpy as np
%matplotlib inline